<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Modular_Hall_MHD_2D_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modular Hall‑MHD 2D (with Arithmetic Regulators)
This Colab‑ready notebook runs a **2D incompressible Hall‑MHD** toy simulator with two modular regulators:
- **Hecke‑like spectral mixing** `Tn` (tiny SL(2,ℤ) stencil in k‑space)
- **Serre‑like drift** `Dk` (phase shear + mild scale‑aware damping)

Outputs: energies over time and magnetic field snapshots.


## 1) Setup

In [ ]:
# If running on Google Colab, NumPy/Matplotlib are typically preinstalled.
# Uncomment if needed:
# !pip -q install numpy scipy matplotlib
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
from math import pi

## 2) Parameters

In [ ]:
# Grid & physics
nx, ny = 256, 256
Lx, Ly = 2*pi, 2*pi
nu, eta = 1e-4, 1e-4          # viscosity, resistivity
di = 0.1                      # ion skin depth (Hall strength)

# Modular regulators (start small)
alpha = 0.02   # on u: Hecke-like mixing
beta  = 0.02   # on u: Serre-like drift
lam   = 0.02   # on B: Hecke-like mixing
eps   = 0.02   # on B: Serre-like drift

stencil = 5    # modular stencil size in k-space: 3 or 5
seed = 7

# Time
steps = 800
dt = None      # None => adaptive Hall-aware CFL
save_every = 0 # set >0 to save snapshots as npz


## 3) Core numerics (spectral operators & regulators)

In [ ]:
def kspace(nx, ny, Lx, Ly):
    kx = fft.fftfreq(nx, d=Lx/nx) * 2*np.pi
    ky = fft.fftfreq(ny, d=Ly/ny) * 2*np.pi
    KX, KY = np.meshgrid(kx, ky, indexing='ij')
    K2 = KX*KX + KY*KY
    K2[0,0] = 1.0
    return KX, KY, K2

def project_div_free(Fx_hat, Fy_hat, KX, KY, K2):
    KdotF = KX*Fx_hat + KY*Fy_hat
    Fx_hat_proj = Fx_hat - (KdotF / K2) * KX
    Fy_hat_proj = Fy_hat - (KdotF / K2) * KY
    Fx_hat_proj[0,0] = 0.0
    Fy_hat_proj[0,0] = 0.0
    return Fx_hat_proj, Fy_hat_proj

def grad(f, KX, KY):
    f_hat = fft.fft2(f)
    dfdx = np.real(fft.ifft2(1j*KX*f_hat))
    dfdy = np.real(fft.ifft2(1j*KY*f_hat))
    return dfdx, dfdy

def grad_from_scalar(phi, KX, KY):
    phi_hat = fft.fft2(phi)
    dphidx_hat = 1j*KX*phi_hat
    dphidy_hat = 1j*KY*phi_hat
    return np.real(fft.ifft2(dphidy_hat)), np.real(fft.ifft2(-dphidx_hat))

def hecke_like_mix(Fx_hat, Fy_hat, stencil, KX, KY):
    nx, ny = KX.shape
    kx = KX[:,0]
    ky = KY[0,:]
    def idx_from_k(k, grid):
        return int(np.clip(np.argmin(np.abs(grid - k)), 0, grid.size-1))

    def sample_from_map(Fx_hat, Fy_hat, mapfunc):
        outFx = np.zeros_like(Fx_hat, dtype=np.complex128)
        outFy = np.zeros_like(Fy_hat, dtype=np.complex128)
        for i in range(nx):
            for j in range(ny):
                kxi, kyj = KX[i,j], KY[i,j]
                kx_m, ky_m = mapfunc(kxi, kyj)
                ii = idx_from_k(kx_m, kx)
                jj = idx_from_k(ky_m, ky)
                outFx[i,j] = Fx_hat[ii,jj]
                outFy[i,j] = Fy_hat[ii,jj]
        return outFx, outFy

    I  = lambda kx,ky: (kx,ky)
    S  = lambda kx,ky: (-ky, kx)
    T  = lambda kx,ky: (kx+ky, ky)
    TS = lambda kx,ky: (kx+2*ky, ky)
    ST = lambda kx,ky: (-ky + kx, kx)
    stencil_maps = [I, S, T] if stencil == 3 else [I, S, T, TS, ST]

    accFx = np.zeros_like(Fx_hat, dtype=np.complex128)
    accFy = np.zeros_like(Fy_hat, dtype=np.complex128)
    for mp in stencil_maps:
        Fx_m, Fy_m = sample_from_map(Fx_hat, Fy_hat, mp)
        accFx += Fx_m
        accFy += Fy_m
    count = len(stencil_maps)
    return accFx / count, accFy / count

def serre_drift(F_hat, KX, KY, kappa=0.2, c0=0.01):
    K2 = KX*KX + KY*KY
    K = np.sqrt(K2)
    kmax = np.max(K)
    mask = (K > 0)
    Omega = np.zeros_like(K)
    c = np.zeros_like(K)
    Omega[mask] = kappa * (KX[mask]*KY[mask]) / K[mask]
    c[mask] = c0 * (K[mask] / kmax)
    return 1j*Omega*F_hat - c*F_hat

def compute_dt(ux, uy, Bx, By, di, KX, KY, CFL=0.3):
    K = np.sqrt(KX*KX + KY*KY)
    kmax = np.max(K)
    umax = max(np.abs(ux).max(), np.abs(uy).max())
    Bmax = max(np.abs(Bx).max(), np.abs(By).max())
    denom = 1e-12 + umax * kmax + di * Bmax * (kmax**2)
    return CFL / denom


## 4) Initial condition (magnetic island / X‑point)

In [ ]:
KX, KY, K2 = kspace(nx, ny, Lx, Ly)

x = np.linspace(0, Lx, nx, endpoint=False)
y = np.linspace(0, Ly, ny, endpoint=False)
X, Y = np.meshgrid(x, y, indexing='ij')
psi = np.cos(X) * np.cos(Y)
Bx0, By0 = grad_from_scalar(psi, KX, KY)

ux0 = 0.2 * np.sin(Y)
uy0 = np.zeros_like(ux0)

rng = np.random.default_rng(seed)
Bx0 = Bx0 + 1e-3 * rng.standard_normal(Bx0.shape)
By0 = By0 + 1e-3 * rng.standard_normal(By0.shape)

ux_hat = fft.fft2(ux0); uy_hat = fft.fft2(uy0)
Bx_hat = fft.fft2(Bx0); By_hat = fft.fft2(By0)

print("IC ready.")


IC ready.


## 5) Time integration

In [ ]:
t = 0.0
if dt is None:
    dt = compute_dt(np.real(fft.ifft2(ux_hat)),
                    np.real(fft.ifft2(uy_hat)),
                    np.real(fft.ifft2(Bx_hat)),
                    np.real(fft.ifft2(By_hat)),
                    di, KX, KY, CFL=0.3)

Ekin_hist, Emag_hist, t_hist = [], [], []

for step in range(1, steps+1):
    ux = np.real(fft.ifft2(ux_hat)); uy = np.real(fft.ifft2(uy_hat))
    Bx = np.real(fft.ifft2(Bx_hat)); By = np.real(fft.ifft2(By_hat))

    if step % 10 == 1:
        dt = compute_dt(ux, uy, Bx, By, di, KX, KY, CFL=0.3)

    dux_dx, dux_dy = grad(ux, KX, KY)
    duy_dx, duy_dy = grad(uy, KX, KY)
    dBx_dx, dBx_dy = grad(Bx, KX, KY)
    dBy_dx, dBy_dy = grad(By, KX, KY)

    conv_x = ux*dux_dx + uy*dux_dy
    conv_y = ux*duy_dx + uy*duy_dy

    Jz = dBy_dx - dBx_dy
    Lorentz_x = Jz * By
    Lorentz_y = - Jz * Bx

    NLux = -conv_x + Lorentz_x
    NLuy = -conv_y + Lorentz_y

    EMFz = ux*By - uy*Bx
    dEMFz_dx = np.real(fft.ifft2(1j*KX*fft.fft2(EMFz)))
    dEMFz_dy = np.real(fft.ifft2(1j*KY*fft.fft2(EMFz)))
    RHS_Bx_real = dEMFz_dy
    RHS_By_real = -dEMFz_dx

    RHS_ux = fft.fft2(NLux)
    RHS_uy = fft.fft2(NLuy)
    RHS_Bx = fft.fft2(RHS_Bx_real)
    RHS_By = fft.fft2(RHS_By_real)

    Tn_ux, Tn_uy = hecke_like_mix(ux_hat, uy_hat, stencil, KX, KY)
    Tn_Bx, Tn_By = hecke_like_mix(Bx_hat, By_hat, stencil, KX, KY)

    Dk_ux = serre_drift(ux_hat, KX, KY, kappa=0.2, c0=0.01)
    Dk_uy = serre_drift(uy_hat, KX, KY, kappa=0.2, c0=0.01)
    Dk_Bx = serre_drift(Bx_hat, KX, KY, kappa=0.2, c0=0.01)
    Dk_By = serre_drift(By_hat, KX, KY, kappa=0.2, c0=0.01)

    RHS_ux += alpha*(Tn_ux - ux_hat) + beta*Dk_ux
    RHS_uy += alpha*(Tn_uy - uy_hat) + beta*Dk_uy
    RHS_Bx += lam*(Tn_Bx - Bx_hat) + eps*Dk_Bx
    RHS_By += lam*(Tn_By - By_hat) + eps*Dk_By

    denom_u = (1.0 + dt*nu*(K2))
    denom_B = (1.0 + dt*eta*(K2))

    ux_hat = (ux_hat + dt*RHS_ux) / denom_u
    uy_hat = (uy_hat + dt*RHS_uy) / denom_u
    Bx_hat = (Bx_hat + dt*RHS_Bx) / denom_B
    By_hat = (By_hat + dt*RHS_By) / denom_B

    ux_hat, uy_hat = project_div_free(ux_hat, uy_hat, KX, KY, K2)
    Bx_hat, By_hat = project_div_free(Bx_hat, By_hat, KX, KY, K2)

    t += dt

    if step % 20 == 0:
        ux = np.real(fft.ifft2(ux_hat)); uy = np.real(fft.ifft2(uy_hat))
        Bx = np.real(fft.ifft2(Bx_hat)); By = np.real(fft.ifft2(By_hat))
        Ekin = 0.5*np.mean(ux**2 + uy**2)
        Emag = 0.5*np.mean(Bx**2 + By**2)
        Ekin_hist.append(Ekin); Emag_hist.append(Emag); t_hist.append(t)
        print(f"step {step:5d}  t={t:7.4f}  Ekin={Ekin:.3e}  Emag={Emag:.3e}  dt={dt:.2e}")

print("Done.")


step    20  t= 0.0018  Ekin=9.999e-03  Emag=2.500e-01  dt=9.05e-05
step    40  t= 0.0036  Ekin=9.999e-03  Emag=2.500e-01  dt=9.05e-05
step    60  t= 0.0054  Ekin=9.998e-03  Emag=2.500e-01  dt=9.05e-05
step    80  t= 0.0072  Ekin=9.998e-03  Emag=2.499e-01  dt=9.05e-05
step   100  t= 0.0090  Ekin=9.998e-03  Emag=2.499e-01  dt=9.05e-05
step   120  t= 0.0109  Ekin=9.997e-03  Emag=2.499e-01  dt=9.05e-05
step   140  t= 0.0127  Ekin=9.997e-03  Emag=2.499e-01  dt=9.04e-05
step   160  t= 0.0145  Ekin=9.997e-03  Emag=2.499e-01  dt=9.04e-05
step   180  t= 0.0163  Ekin=9.996e-03  Emag=2.499e-01  dt=9.04e-05
step   200  t= 0.0181  Ekin=9.996e-03  Emag=2.499e-01  dt=9.04e-05
step   220  t= 0.0199  Ekin=9.996e-03  Emag=2.498e-01  dt=9.03e-05
step   240  t= 0.0217  Ekin=9.996e-03  Emag=2.498e-01  dt=9.03e-05
step   260  t= 0.0235  Ekin=9.997e-03  Emag=2.498e-01  dt=9.02e-05
step   280  t= 0.0253  Ekin=9.997e-03  Emag=2.498e-01  dt=9.02e-05
step   300  t= 0.0271  Ekin=9.998e-03  Emag=2.498e-01  dt=9.01

## 6) Plots — energies and field snapshots

In [ ]:
import matplotlib.pyplot as plt

# Energies
plt.figure(figsize=(6,4))
plt.plot(t_hist, Ekin_hist, label='E_kin')
plt.plot(t_hist, Emag_hist, label='E_mag')
plt.xlabel('t'); plt.ylabel('Energy')
plt.legend()
plt.title('Energies vs time')
plt.show()

# Snapshot: |B| and stream of B
Bx = np.real(fft.ifft2(Bx_hat)); By = np.real(fft.ifft2(By_hat))
Bmag = np.sqrt(Bx*Bx + By*By)

plt.figure(figsize=(6,5))
plt.imshow(Bmag.T, origin='lower', extent=[0,Lx,0,Ly], aspect='equal')
plt.colorbar(label='|B|')
plt.title('Magnetic field magnitude |B|')
plt.xlabel('x'); plt.ylabel('y')
plt.show()

# Streamplot (downsample for speed)
plt.figure(figsize=(6,5))
x = np.linspace(0, Lx, 64); y = np.linspace(0, Ly, 64)
Xg, Yg = np.meshgrid(x, y, indexing='ij')
ix = (np.linspace(0, nx-1, 64)).astype(int)
iy = (np.linspace(0, ny-1, 64)).astype(int)
plt.streamplot(Xg, Yg, Bx[ix][:,iy], By[ix][:,iy], density=1.2)
plt.title('B streamlines (downsampled)')
plt.xlabel('x'); plt.ylabel('y')
plt.axis('equal')
plt.show()


NameError: name 't_hist' is not defined

<Figure size 600x400 with 0 Axes>